**Example of Training CNN with cifar10 Dataset**
---
The data and results will be stored in the following directories

  1. drive/My Drive/public/data/ has data
  2. drive/My Drive/public/results/ will have results



mount your drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


import pytorch

In [ ]:
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable

import warnings
warnings.filterwarnings('ignore')

model definition

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(3,16,3,padding=1),
            nn.ReLU(),
            nn.Conv2d(16,32,3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2), # 32 x 16 x 16
            
            nn.Conv2d(32,64,3,padding=1),
            nn.ReLU(),
            nn.Conv2d(64,128,3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2), # 128 x 8 x 8
            
            nn.Conv2d(128,256,3,padding=1),
            nn.ReLU(),
            nn.Conv2d(256,256,3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )

        conv_size = self.get_conv_size((3,32,32))

        self.fc_layer = nn.Sequential(
            nn.Linear(conv_size,200),
            nn.ReLU(),
            nn.Linear(200,10)
        )       

    def get_conv_size(self, shape):
        o = self.layer(torch.zeros(1, *shape))
        return int(np.prod(o.size()))
        
    def forward(self,x):
        #TODO: define forward function of the model#
        batch_size, c, h, w = x.data.size()
        out = self.layer(x)
        out = out.view(batch_size, -1)
        out = self.fc_layer(out)
        ############################################
        return out


define hyper-parameters (including device)

In [ ]:
learning_rate = 0.0002
num_epoch = 10
batch_size = 32

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data download and data loader

In [ ]:
cifar_train = dset.CIFAR10("drive/My Drive/public/data/", train=True, 
                           transform=transforms.ToTensor(), 
                           target_transform=None, download=True)
cifar_test = dset.CIFAR10("drive/My Drive/public/data/", train=False, 
                          transform=transforms.ToTensor(), 
                          target_transform=None, download=True)

print(cifar_train.__getitem__(0)[0].size(), cifar_train.__len__())
cifar_test.__getitem__(0)[0].size(), cifar_test.__len__()

train_loader = torch.utils.data.DataLoader(cifar_train,batch_size=batch_size, 
                                  shuffle=True,num_workers=2,drop_last=True)
test_loader = torch.utils.data.DataLoader(cifar_test,batch_size=batch_size, 
                                  shuffle=False,num_workers=2,drop_last=True)

Extracting drive/My Drive/public/data/cifar-10-python.tar.gz to drive/My Drive/public/data/
Files already downloaded and verified
torch.Size([3, 32, 32]) 50000


In [ ]:




model = CNN().to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

model and objective function and optimization

---
training 

In [ ]:
try:
    model = torch.load('drive/My Drive/public/results/cifar10_pretrained.pth')
    print("model restored")
except:
    print("model not restored")
    
if not os.path.exists('drive/My Drive/public/results'):
    os.mkdir('drive/My Drive/public/results')    
    
for i in range(num_epoch):
    for j,[image,label] in enumerate(train_loader):
        x = Variable(image).to(device)
        y_= Variable(label).to(device)
        
        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output,y_)
        loss.backward()
        optimizer.step()
        
        if j % 500 == 0:
            print(loss.data,i,j)
    
    torch.save(model,'drive/My Drive/public/results/cifar10_model.pth')
            
param_list = list(model.children())
print(param_list)

model not restored
tensor(2.3126, device='cuda:0') 0 0
tensor(1.8621, device='cuda:0') 0 500


testing

In [ ]:
correct = 0
total = 0

with torch.no_grad():
  for image,label in test_loader:
    x = Variable(image).to(device)
    y_= Variable(label).to(device)

    output = model.forward(x)
    _,output_index = torch.max(output,1)
        
    total += label.size(0)
    correct += (output_index == y_).sum().float()
    
print("Accuracy of Test Data: {}".format(100*correct/total))

torch.save(model, 'drive/My Drive/public/results/cifar10_pretrained.pth')


Accuracy of Test Data: 73.34735870361328


the end!
----